In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from random import randint

In [2]:
import xlrd

In [3]:
data = pd.read_excel('pone.0216416.s003.xlsx')

In [4]:
data = data.drop(['no'], axis=1)

Filling missing values

In [5]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [6]:
iterative_imp = IterativeImputer()

In [7]:
iterative_imp.fit(data)

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=None,
                 sample_posterior=False, skip_complete=False, tol=0.001,
                 verbose=0)

In [8]:
data1 = iterative_imp.fit_transform(data)

In [9]:
data1 = pd.DataFrame(data=data1, columns= data.columns,)

In [10]:
data1.columns = ['sex', 'age', 'duration_of_diabetes', 'BMI',
       'insulin_regimen', 'TDD', 'TDD/kg', 'basal', 'basal/kg',
       'bolus', 'bolus/kg', 'HbA1c', 'eGFR', 'bodyfat', 'adiponectin',
       'free-test', 'SMI', 'grip_strength', 'knee_extension_strength',
       'gait_speed', 'ucOC', 'OC']

# ADDED_WEIGHT


In [11]:
data1['TDDkg'] = data1['TDD/kg']

In [12]:
data1['ADDED_WEIGHT'] = data1.TDD / data1.TDDkg

In [13]:
data1 = data1.drop(['basal/kg', 'bolus/kg', 'TDD/kg', 'TDDkg'], axis = 1)

# Feature Ranking


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = data1.drop('duration_of_diabetes', axis = 1)
y = data1['duration_of_diabetes']

In [16]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.30, random_state= 42)

In [17]:
import gabrielrfe as rfe

In [20]:
ranking = rfe.RankingRE(X, y, 1000)

In [21]:
ranking.ranking_borda()

,Categories,Borda-Score
2,BMI,10561.0
13,grip_strength,10579.0
0,sex,10904.0
11,free-test,10910.0
17,OC,11040.0
5,basal,11139.0
9,bodyfat,11183.0
7,HbA1c,11219.0
10,adiponectin,11221.0
3,insulin_regimen,11295.0
